In [1]:
import tensorflow as tf
from datetime import datetime

%load_ext tensorboard

# 构建tf model
1、所有的model(layer)都继承tf.Module类

In [4]:
# eager execute
# 动态图
class SimpleModule(tf.Module):
    def __init__(self, name=None): # 模型初始化，定义所需变量和常量
        super().__init__(name=name)
        self.a_variable = tf.Variable(5.0, name='train_me')
        self.non_trainable_varibale = tf.Variable(5.0, trainable=False, name="do_not_train_me")
    def __call__(self, x): # 前向过程
        return self.a_variable * x + self.non_trainable_varibale

simple_module = SimpleModule(name="simple")
simple_module(tf.constant(5.0))

<tf.Tensor: shape=(), dtype=float32, numpy=30.0>

如何获得静态图？
用tf.function修饰函数，修饰器会捕捉函数中的所有tf.Op和python logic，构成计算图（概念中的）。第一次调用该函数的时候，会执行trace，在内存中真正创建tf.Graph（tf.Autograph）  
在模型的class代码中，用tf.function修饰前向过程（即__call__函数）。第一次开始前向过程的时候，会执行trace，构建tf.Graph  
可以使用  tfmodel.__call__.get_concrete_function(input_data).graph.as_graph_def()  的方法，打印计算图

In [13]:
# graph execute
# 静态图
class SimpleModule(tf.Module):
    def __init__(self, name=None):
        super().__init__(name=name)
        self.a_variable = tf.Variable(5.0, name='train_me')
        self.non_trainable_varibale = tf.Variable(5.0, trainable=False, name="do_not_train_me")

    @tf.function
    def __call__(self, x): # tf.function修饰前向过程，得到静态图的模型
        return self.a_variable * x + self.non_trainable_varibale

simple_module = SimpleModule(name='simplegraph')

In [17]:
# 打印图结构
simple_module.__call__.get_concrete_function(tf.constant(5.0)).graph.as_graph_def()

node {
  name: "x"
  op: "Placeholder"
  attr {
    key: "_user_specified_name"
    value {
      s: "x"
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "shape"
    value {
      shape {
      }
    }
  }
}
node {
  name: "ReadVariableOp/resource"
  op: "Placeholder"
  attr {
    key: "dtype"
    value {
      type: DT_RESOURCE
    }
  }
  attr {
    key: "shape"
    value {
      shape {
      }
    }
  }
}
node {
  name: "ReadVariableOp"
  op: "ReadVariableOp"
  input: "ReadVariableOp/resource"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
}
node {
  name: "mul"
  op: "Mul"
  input: "ReadVariableOp"
  input: "x"
  attr {
    key: "T"
    value {
      type: DT_FLOAT
    }
  }
}
node {
  name: "add/ReadVariableOp/resource"
  op: "Placeholder"
  attr {
    key: "dtype"
    value {
      type: DT_RESOURCE
    }
  }
  attr {
    key: "shape"
    value {
      shape {
      }
    }
  }
}
node {
  name: "add/ReadVariab

2、用model构建model

In [20]:
class Dense(tf.Module): # dense layer
    def __init__(self, in_features, out_features, name=None):
        super().__init__(name=name)
        self.w = tf.Variable(tf.random.normal([in_features, out_features]), name='w')
        self.b = tf.Variable(tf.zeros(out_features), name='b')
    def __call__(self, x):
        y = tf.matmul(x, self.w) + self.b
        return tf.nn.relu(y)

class SequentialModule(tf.Module):
    def __init__(self, name=None):
        super().__init__(name=name)
        self.dense_1 = Dense(in_features=3, out_features=3)
        self.dense_2 = Dense(in_features=3, out_features=2)
    
    @tf.function
    def __call__(self, x):
        x = self.dense_1(x)
        return self.dense_2(x)

my_model = SequentialModule(name="mlp_2layer")

In [23]:
my_model.__call__.get_concrete_function(tf.constant([[1., 2., 3.]])).graph.as_graph_def()

node {
  name: "x"
  op: "Placeholder"
  attr {
    key: "_user_specified_name"
    value {
      s: "x"
    }
  }
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "shape"
    value {
      shape {
        dim {
          size: 1
        }
        dim {
          size: 3
        }
      }
    }
  }
}
node {
  name: "MatMul/ReadVariableOp/resource"
  op: "Placeholder"
  attr {
    key: "dtype"
    value {
      type: DT_RESOURCE
    }
  }
  attr {
    key: "shape"
    value {
      shape {
      }
    }
  }
}
node {
  name: "MatMul/ReadVariableOp"
  op: "ReadVariableOp"
  input: "MatMul/ReadVariableOp/resource"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
}
node {
  name: "MatMul"
  op: "MatMul"
  input: "x"
  input: "MatMul/ReadVariableOp"
  attr {
    key: "T"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "transpose_a"
    value {
      b: false
    }
  }
  attr {
    key: "transpose_b"
    value {
      b: f

tf.Module类有类方法，可以展示它自动收集的tf.Variable实例和tf.Module实例。初始化一个实例之后就可以看到。

In [21]:
my_model.submodules

(<__main__.Dense at 0x7f84cbc1d250>, <__main__.Dense at 0x7f84cabe1e80>)

In [22]:
for var in my_model.variables:
    print(var, '\n')

<tf.Variable 'b:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)> 

<tf.Variable 'w:0' shape=(3, 3) dtype=float32, numpy=
array([[ 0.6255698 , -0.7725213 ,  0.7988336 ],
       [-1.8342514 , -0.11878946, -0.5036815 ],
       [ 0.07021609, -0.9568012 ,  0.2326703 ]], dtype=float32)> 

<tf.Variable 'b:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)> 

<tf.Variable 'w:0' shape=(3, 2) dtype=float32, numpy=
array([[0.22425525, 1.1546009 ],
       [1.2926105 , 1.4845854 ],
       [0.00476493, 0.6000823 ]], dtype=float32)> 



In [19]:
# all trainable variables
simple_module.trainable_variables

(<tf.Variable 'train_me:0' shape=() dtype=float32, numpy=5.0>,)

In [18]:
# all variables
simple_module.variables

(<tf.Variable 'train_me:0' shape=() dtype=float32, numpy=5.0>,
 <tf.Variable 'do_not_train_me:0' shape=() dtype=float32, numpy=5.0>)

3、延迟构建（deferr）  
即不是在初始化实例的时候构建内部参数变量，而是在第一次前向过程的时候，构建内部变量  
好处是等拿到input之后，才开始构建内部参数变量。所以内部参数变量可以根据第一次input来创建